In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Standard library imports.
import datetime
import itertools
import json

# Related third party imports.
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format
import tqdm

# Local application/library specific imports.
from src.models.network_simulation import NetworkSimulation
from src.data.json_encoder import NumpyEncoder

def add_cumulative(df):
    for sim_name in df['simulation_name'].unique():
        for sim_id in df['simulation_id'].unique():
            mask = (df['simulation_id'] == sim_id) & (df['simulation_name'] == sim_name)
            df.loc[mask,'acceptable_count_cum'] = df.loc[mask,'acceptable_count'].cumsum()
            df.loc[mask,'vSDN_count_cum'] = df.loc[mask,'vSDN_count'].cumsum()
            df.loc[mask,'acceptance_ratio_cum'] = df.loc[mask,'acceptable_count_cum'] / df.loc[mask,'vSDN_count_cum']

In [27]:
print(datetime.datetime.now().strftime("%Y-%m-%d-%H-%M"))

2021-11-15-19-58


In [ ]:
network_name = '25_italy'
max_vSDN_size = 10
hp_type = 'ilp'
hp_objective = 'acceptance ratio'

possible_settings = {
    'network_name': [network_name],
    'latency_factor': [0.5], # np.arange(0.4, 0.7, 0.1),
    'shortest_k': [16],
    'hp_type': [hp_type],
    'hp_objective': [hp_objective],
    'sim_repeat': [3],
    'timesteps': [10],
    'max_request_size': [max_vSDN_size],
    'request_per_timestep': [5,6],
    'TTL_range': [5],
    'vSDN_count_ilp':[10],
    'vSDN_size_ilp':[max(2, int(max_vSDN_size / 2))]
}
param_names_1 = list(possible_settings.keys())
setting_list = [
    dict(zip(param_names_1, x))
    for x in itertools.product(*possible_settings.values())
]

In [ ]:
# Normal Simulation
setting = setting_list[0]

ns = NetworkSimulation(**setting)
ns.init_simulation(**setting)
ns.run_multiple_dynamic_simulations(**setting)
run_log = ns.get_logs()
df = pd.DataFrame(run_log)
add_cumulative(df)

In [ ]:
df[['hp_runtime', 'acceptance_ratio', 'acceptance_ratio_cum', 'active_vSDN_count', 'mean_switch_hpair_count', 'simulation_id', 'timestep', 'max_vSDN_size']]

In [ ]:
df.groupby(by=['simulation_name', 'timestep']).mean()[['acceptance_ratio_cum', 'active_vSDN_count']]

In [ ]:
# Simulation
simulation_logs = []
for setting in tqdm.tqdm(setting_list, total=len(setting_list)):
    ns = NetworkSimulation(**setting)
    ns.init_simulation(**setting)
    ns.run_multiple_dynamic_simulations(**setting)
    simulation_logs.extend(ns.get_logs())
dfs = pd.DataFrame(simulation_logs)
add_cumulative(dfs)

In [ ]:
dfs.shape

In [ ]:
dfs[['hp_runtime', 'acceptance_ratio', 'acceptance_ratio_cum', 'active_vSDN_count', 'mean_switch_hpair_count', 'simulation_id', 'timestep', 'max_vSDN_size', 'vSDN_count']]

In [30]:
dfs.groupby(by=['simulation_name', 'timestep']).mean()[['acceptance_ratio_cum', 'active_vSDN_count']]

acceptance_ratio_cum  active_vSDN_count
simulation_name              timestep                                         
25_italy_L50_mrs10_rpt5_ttl5 1                         0.60               3.00
                             2                         0.43               4.00
                             3                         0.42               4.00
                             4                         0.40               4.33
                             5                         0.37               4.67
                             6                         0.34               3.00
                             7                         0.35               4.67
                             8                         0.33               4.00
                             9                         0.33               2.67
                             10                        0.31               3.33
25_italy_L50_mrs10_rpt6_ttl5 1                         0.33               2.00
                             2                         0.25               1.67
                             3                         0.26               2.67
                             4                         0.25               3.33
                             5                         0.23               2.33
                             6                         0.28               4.00
                             7                         0.27               3.33
                             8                         0.27               3.67
                             9                         0.27               4.67
                             10                        0.28               4.00

In [ ]:
# Optimal Run
setting = setting_list[0]

ns = NetworkSimulation(**setting)
ns.init_simulation(**setting)
ns.run_optimal_dynamic_simulation(**setting)
run_log_opt = ns.get_logs()
df_opt = pd.DataFrame(run_log_opt)
add_cumulative(df_opt)

In [ ]:
df_opt[['hp_runtime', 'acceptance_ratio', 'acceptance_ratio_cum', 'active_vSDN_count', 'mean_switch_hpair_count', 'hp_changed']]

In [ ]:
pd.concat([df[['hp_runtime', 'acceptance_ratio_cum', 'active_vSDN_count', 'mean_switch_hpair_count', 'h_list']],df_opt[['hp_runtime', 'acceptance_ratio_cum', 'active_vSDN_count', 'mean_switch_hpair_count', 'h_list', 'hp_changed']]], axis=1).sort_index(axis=1)

In [ ]:
# Baseline Simulation
simulation_logs = []
for setting in tqdm.tqdm(setting_generator, total=len(setting_generator)):
    ns = NetworkSimulation(**setting)
    ns.init_simulation(**setting)
    ns.baseline_simulation(possible_request_settings=possible_request_settings, **setting)
    simulation_logs.extend(ns.get_logs())

In [ ]:
with open(f"../results/{network_name}/{datetime.date.today()}-{network_name}-test.json", 'w') as file:
    json.dump(simulation_logs, file, indent=4, sort_keys=True,
              separators=(', ', ': '), ensure_ascii=False,
              cls=NumpyEncoder)

# Analyse simulation logs

In [ ]:
df_heu = pd.DataFrame(simulation_logs)

In [ ]:
df_heu.query("latency_factor == 0.5 & vSDN_size == 2")[['hp_runtime', 'acceptance_ratio']]#.mean()

In [ ]:
df_ilp = pd.DataFrame(simulation_logs)

In [ ]:
df_ilp.query("latency_factor == 0.4 & vSDN_size == 3")[['hp_runtime', 'acceptable_count','acceptance_ratio', 'acceptance_ratio_ilp', 'h_list']]#.mean()

In [ ]:
df_opt = pd.DataFrame(simulation_logs)

In [ ]:
df_opt.query("latency_factor == 0.4 & vSDN_size == 2")[['hp_runtime', 'acceptance_ratio', 'acceptance_ratio_ilp']]#.mean()